# Action2  对Titanic数据进行清洗，建模并对乘客生存进行预测。使用之前介绍过的10种模型中的至少2种（包括TPOT）  
1、完成代码，结果正确（20points）     
2、能使用TPOT完成预测（20points）  

PassengerId 用户编号：
记录乘客的Id编号。经过了解后：

并没有查到其构成具有特别的实际意义（如身份证的构成每一位都是有实际意义的）；
仅作为唯一标识来定位到某一乘客身上（唯一值同总数据量一样）；
因此认为不具有分析的价值，过后也会将它进行删除处理。

Survived 是否存活（label）：
描述乘客是否存活。

0 - 用户未能存活；
1- 用户存活；
Pclass（用户阶级）：
描述用户所属的等级，总共分为三等，用1、2、3来描述，其中：

1 - 1st class，高等用户；
2 - 2nd class，中等用户；
3 - 3rd class，低等用户；
Name（名字）：
描述乘客的全名。例如上例中的 Rugg, Miss. Emily 中：

Rugg ：first name，即名；
Miss. ：title，即称谓；
Emily ：last name，即姓
在登记乘客姓名时全都是用这种方法进行记录的；

Sex（性别）：
描述乘客的性别，其中：

male - 男性；
female - 女性；
Age（年龄）：
描述乘客的年龄，其中有部分缺失值，需要用一些手段将她们补全，具体的方法方在下面数据清洗中；

SibSp 和 Parch：
SibSp：描述了泰坦尼克号上与乘客同行的兄弟姐妹（Siblings）和配偶（Spouse）数目；
Parch：描述了泰坦尼克号上与乘客同行的家长（Parents）和孩子（Children）数目；
Ticket（船票号）：
描述乘客登船所使用的船票编号。虽然它没有编码上的规律，不存在缺失值，但是唯一值可以看到，同之前唯一定位的乘客编号不同，也就是说可能会有人重复使用船票的情况，具体处理会在数据清洗中介绍，我会找到资料支撑和这一想法；

Fare（乘客费用）：
描述乘客上传所花费的费用；

Cabin（船舱）：
描述用户所住的船舱编号。由两部分组成，仓位号和房间编号，如C88中，C和88分别对应C仓位和88号房间。本字段缺失值较多，具体处理方法会在后面的数据清洗部分进行介绍。

Embarked（港口）：
描述乘客上船时的港口，包含三种类型：

C：Cherbourg；
Q：Queenstown；
S：Southampton；

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer #特征提取
from sklearn.model_selection import cross_val_score
from sklearn import metrics

In [2]:
# 数据加载
train_data =pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')

In [7]:
# 数据探索
print('查看数据信息：列名、非空个数、类型等')
print(train_data.info())

查看数据信息：列名、非空个数、类型等
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None


In [8]:
print('查看数据摘要')
train_data.describe()

查看数据摘要


,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [9]:
print('查看离散数据分布')
train_data.describe(include=['O'])

查看离散数据分布


,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Bryhl, Mr. Kurt Arnold Gottfrid",male,CA. 2343,G6,S
freq,1,577,7,4,644


In [10]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [16]:
# 使用平均年龄来填充年龄中的nan值
train_data['Age'].fillna(train_data['Age'].mean(), inplace=True)
test_data['Age'].fillna(test_data['Age'].mean(),inplace=True)

In [18]:
# 使用票价的均值填充票价中的nan值
train_data['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
test_data['Fare'].fillna(test_data['Fare'].mean(),inplace=True)

In [20]:
print(train_data['Embarked'].value_counts())
# 使用登录最多的港口来填充登录港口的nan值
train_data['Embarked'].fillna('S', inplace=True)
test_data['Embarked'].fillna('S',inplace=True)

S    644
C    168
Q     77
Name: Embarked, dtype: int64


In [24]:
# 特征选择
features = ['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare', 'Embarked']
train_features = train_data[features]
train_labels = train_data['Survived']
test_features = test_data[features]
train_features

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.000000,1,0,7.2500,S
1,1,female,38.000000,1,0,71.2833,C
2,3,female,26.000000,0,0,7.9250,S
3,1,female,35.000000,1,0,53.1000,S
4,3,male,35.000000,0,0,8.0500,S
...,...,...,...,...,...,...,...
886,2,male,27.000000,0,0,13.0000,S
887,1,female,19.000000,0,0,30.0000,S
888,3,female,29.699118,1,2,23.4500,S
889,1,male,26.000000,0,0,30.0000,C


In [27]:
# 特征提取
# 'records' : list like [{column -> value}, ... , {column -> value}]
dvec=DictVectorizer(sparse=False)
train_features = dvec.fit_transform(train_features.to_dict(orient='record'))
train_features

array([[22.        ,  0.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ],
       [38.        ,  1.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [26.        ,  0.        ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [29.69911765,  0.        ,  0.        , ...,  1.        ,
         0.        ,  1.        ],
       [26.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  0.        ],
       [32.        ,  0.        ,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [32]:
# dvec.get_feature_names()
dvec.feature_names_

['Age',
 'Embarked=C',
 'Embarked=Q',
 'Embarked=S',
 'Fare',
 'Parch',
 'Pclass',
 'Sex=female',
 'Sex=male',
 'SibSp']

In [43]:
# 构造ID3决策树
clf = DecisionTreeClassifier(criterion='entropy')
clf.fit(train_features,train_labels)

DecisionTreeClassifier(criterion='entropy')

In [44]:
# 测试集特征提取转换（已经fit过了不需要再fit）
test_features = dvec.transform(test_features.to_dict(orient='record'))

In [45]:
# 预测
pred_labels = clf.predict(test_features)
acc_decision_tree = round(clf.score(train_features,train_labels),6)
print('score准确率为%.4lf'%acc_decision_tree)

score准确率为0.9820


In [47]:
# 使用k折交叉验证统计决策树准确率
print('cross_val_score准确率为%.4lf'% np.mean(cross_val_score(clf,train_features,train_labels,cv=10)))

cross_val_score准确率为0.7779


In [52]:
# 使用TPOT自动机器学习工具对titanic进行分类
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import numpy as np

tpot = TPOTClassifier(generations=10, population_size=20, verbosity=2)
tpot.fit(train_features,train_labels)

Optimization Progress: 100%|███████████████████████████████████████████████████| 220/220 [04:18<00:00,  1.33s/pipeline]
                                                                                                                       
Best pipeline: RandomForestClassifier(XGBClassifier(SelectFromModel(input_matrix, criterion=gini, max_features=1.0, n_estimators=100, threshold=0.0), learning_rate=0.5, max_depth=6, min_child_weight=16, n_estimators=100, nthread=1, subsample=0.15000000000000002), bootstrap=False, criterion=gini, max_features=0.4, min_samples_leaf=4, min_samples_split=11, n_estimators=100)


TPOTClassifier(generations=10,
               log_file=<ipykernel.iostream.OutStream object at 0x00000233C84D4D60>,
               population_size=20, verbosity=2)

In [53]:
print('tpot准确率为%.4lf'%tpot.score(train_features,train_labels))

tpot准确率为0.8945


In [56]:
# 创建cart分类器
from sklearn.tree import DecisionTreeClassifier
cart_model = DecisionTreeClassifier(criterion='gini')
cart_model.fit(train_features,train_labels)
pred_labels = cart_model.predict(test_features)
acc_decision_tree = round(cart_model.score(train_features,train_labels),6)
print('cart_model的score准确率为%.4lf'%acc_decision_tree)

cart_model的score准确率为0.9820


In [63]:
# 创建LR分类器
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_features,train_labels)
predict_y=lr.predict(test_features)
print('LR准确率: %0.4lf' % lr.score(train_features,train_labels))

LR准确率: 0.7991


In [65]:
from sklearn.ensemble import RandomForestClassifier 
RF = LogisticRegression()
RF.fit(train_features,train_labels)
predict_y=RF.predict(test_features)
print('RF准确率: %0.4lf' % RF.score(train_features,train_labels))

RF准确率: 0.7991
